<a href="https://colab.research.google.com/github/xHexlabx/SuperAI_SS4_Recap/blob/main/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/Liver_Ultrasound_Detection_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SuperAI Season 4 - Level 2 Hackathon - Liver_Ultrasound_Detection_Classification

## Mounting Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Custom Dataset For Classification

In [2]:
import pandas as pd
from PIL import Image , ImageOps
import numpy as np
import os


In [3]:
train_annotation_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/annotations/annotation_for_train_datasets.csv'
validation_annotation_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/annotations/annotation_for_validation_datasets.csv'

In [4]:
train_annotation_df = pd.read_csv(train_annotation_path)

In [5]:
train_annotation_df.iloc[0]['file_name']

98356

In [6]:
len(train_annotation_df)

14448

In [7]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets , transforms
from torchvision.transforms import ToTensor, Lambda
import matplotlib.pyplot as plt

In [8]:
class UltraSoundDataset (Dataset) :

    def __init__ (self , annotation_path , image_path , transform = None , target_transform = None) :

        self.annotation_path = annotation_path
        self.annotation_file = pd.read_csv(annotation_path)

        self.image_path = image_path

        self.transform = transform
        self.target_transform = target_transform

    def __len__(self) :

        return len(self.annotation_file)

    def __getitem__ (self , idx) :

        image_name = self.annotation_file.iloc[idx]['file_name']

        image = Image.open(f'{self.image_path}/{image_name}.jpg')

        label = self.annotation_file.iloc[idx]['class']

        if self.transform :

            image = self.transform(image)

        if self.target_transform :

            label = self.target_transform(label)

        return image , label


In [9]:
transform = transforms.Compose([
    transforms.Resize((224 , 224)),  # Resize the image to 128x128
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
])

In [10]:
target_transform = Lambda(lambda y: torch.zeros(2, dtype=torch.float).scatter_(dim = 0, index = torch.tensor(y), value = 1))

In [11]:
annotation_for_train_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/annotations/annotation_for_train_datasets.csv'
train_image_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/yolo_format_datasets/train/images'

annotation_for_validation_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/annotations/annotation_for_validation_datasets.csv'
validation_image_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/yolo_format_datasets/validation/images'

In [12]:
from torch.utils.data import DataLoader

In [13]:
train_dataset = UltraSoundDataset(annotation_for_train_path , train_image_path , transform , target_transform)
train_dataloader = DataLoader(train_dataset , batch_size = 16, shuffle = True , num_workers = 8)

validation_dataset = UltraSoundDataset(annotation_for_validation_path , validation_image_path , transform , target_transform)
validation_dataloader = DataLoader(validation_dataset , batch_size = 16, shuffle = False , num_workers = 8)

## Set Varibles For Model Training | MaxVit

In [14]:
!pip install timm
!pip install evaluate

In [15]:
import timm
import timm.optim
import timm.scheduler
from timm.data import ImageDataset, create_dataset, create_loader
from timm.data.transforms_factory import create_transform
from torch import nn

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from PIL import Image

import evaluate

import numpy as np
import pandas as pd
from scipy import stats

from tqdm.notebook import tqdm

In [16]:
!nvidia-smi

Thu May 30 09:23:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   67C    P8              18W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [17]:
num_epochs = 25

model_name = "timm/maxvit_tiny_tf_224.in1k"

criterion = nn.CrossEntropyLoss()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = timm.create_model(model_name, pretrained = True, num_classes = 2).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [26]:
optimizer = timm.optim.create_optimizer_v2(model , opt = "AdamW", lr = 1e-4)
optimizer = timm.optim.Lookahead(optimizer , alpha = 0.5, k = 6)

In [18]:
device

device(type='cuda')

In [27]:
def training (num_epochs , train_dataloader , model , criterion , optimizer) :

    model.train()

    for epoch in range(25 , 25 + num_epochs) :

        print(f'epoch = {epoch}')

        losses = 0.0

        for idx , batch in enumerate(tqdm(train_dataloader)) :

            images , labels = batch
            images , labels = images.to(device) , labels.to(device)

            outputs = model(images.to(device))

            loss = criterion(outputs, labels.to(device))
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()

            losses += loss.item()

            if (idx != 0 and (idx + 1) % 25 == 0 ) :

                print(losses / 25.0)

                losses = 0


        torch.save(model.state_dict(), f'/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/model_checkpoints/maxvit_model_{epoch}.pth')


In [28]:
training(num_epochs , train_dataloader , model , criterion , optimizer)

epoch = 25


  0%|          | 0/903 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


0.04984259937889874
0.03275605026166886
0.03137651581782848
0.030507073514163496
0.03213974517537281
0.022204691348597407
0.02459983026375994
0.04887370212934911
0.017184201690834015
0.03337143682409078
0.03098710426595062
0.016593199658673258
0.007592640360817313
0.026916378089226783
0.018961448955815287
0.023787828772328795
0.012171842416282743
0.014928458830108866
0.022651914116577247
0.021567043545655907
0.01640855724224821
0.015973783610388637
0.013945043308194727
0.007680411750334315
0.0075430744822369885
0.012489494816400111
0.009299795398837886
0.006214006853406317
0.0071743176717427555
0.034885524442070166
0.0034643432090524584
0.0184653500438435
0.029105384999420492
0.038779514961061065
0.015484122544294224
0.017739128888351843
epoch = 26


  0%|          | 0/903 [00:00<?, ?it/s]

...

0.0010625776268807386
0.00022460843738372204
4.703337720457057e-05
5.961577426205622e-05
8.050344828916422e-05
0.0001406276522720873
0.0007162411881654406
2.3280313362192827e-05
2.919353552897519e-05
2.5795127821766073e-05
0.00017396576187820757
0.0007386089297051513
0.0037699474272858426
0.00030775988785535446
0.021357441831078176
0.0006415673614174011
4.653361516830046e-05
0.0003871383044133836
4.7561875071551185e-05
0.0020106994073103126
0.0002640944607082929
8.044241192692425e-05
0.00013487257729138946
0.00025711884121847105
0.0014887496375104092
7.0649565095664e-05
0.0007417682668437919
0.00031021265836898236
0.00019985074720352714
0.0006748569772753399
6.605221350582725e-05
0.00014137089015093808
5.8475975138208013e-05
0.00016557882024244462
0.00021616379212900937
0.0008770682754766313
epoch = 48


  0%|          | 0/903 [00:00<?, ?it/s]

5.8988390537706434e-05
2.7230067971686366e-05
4.434602633409668e-05
2.7646822968563355e-05
0.02138915224424636
0.005417122963463044
0.0001262122395201004
0.000362292677982623
0.0002496881027764175
0.00023703624909103384
0.0018352902548394922
0.00026233875818434173
0.00018784538788167992
3.6633140771300534e-05
3.7677715881727635e-05
3.924526819901075e-05
4.828315974918951e-05
0.00011051816205508658
0.00043508395849130467
0.00014337877117213794
6.948883203222066e-05
1.8629685619089288e-05
0.00013166888809791998
0.0001845593287362135
0.006999176685967541
7.749869287181355e-05
0.0007683826458651311
3.0729923309991136e-05
0.00027701674727723005
5.94243763043778e-05
4.4880584027851e-05
0.00036337398107207265
4.868316982083343e-05
8.351944479727536e-05
2.698473826967529e-05
0.0001710471658498136
epoch = 49


  0%|          | 0/903 [00:00<?, ?it/s]

4.60997950995079e-05
1.5503742779401365e-05
0.0008356199201716663
7.65121307085792e-05
0.00017605941830879602
4.083011592683761e-05
8.321013780914654e-05
0.00019159876400408392
1.5517092028858313e-05
9.295552963521914e-06
0.0001572709112724624
3.9422296003976955e-05
2.5743900851011857e-05
0.00013794315984796412
8.50913373142248e-06
0.00015646719868527725
1.2671881663663953e-05
1.2445405745893368e-05
2.0591806746779183e-05
2.0729214111270266e-05
8.433846610387263e-06
1.6362523178941047e-05
0.00016555824302486144
0.0001561994525627597
7.921362337128813e-06
5.2776855059164516e-05
1.3097751830173366e-05
0.0001386736495715013
0.00015037601678159263
4.5552340166068463e-05
0.0003614260952144832
9.593437343937694e-06
0.0005342356757955713
0.00014343982850959947
1.8292636334535927e-05
6.153388142138283e-06


In [31]:
def evaluating (model , validation_dataloader) :

    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():

        for idx , batch in enumerate(tqdm(validation_dataloader)) :

            images , labels = batch
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _ , predicted = torch.max(outputs.data, 1)

            total += labels.size(0)

            correct += (predicted == labels.argmax(axis = 1).squeeze()).sum().item()

    print(f'Validation Accuracy: {100 * correct / total}%')

In [32]:
evaluating(model , validation_dataloader)

  0%|          | 0/307 [00:00<?, ?it/s]

Validation Accuracy: 83.8505512454063%
